When building a tree, you want each partition to become “purer” (i.e., containing only data from a single class). If your partitions are pure, you can easily and confidently assign labels to new data points that lie within a partition. You use an impurity metric to measure a partition’s purity compared to other partitions. 

Gini impurity:
    0 == all labels are 0
    .5 == half labels are 0, other half are 1
    1 == all are one

Impurity reaches a maximum value for a set of labels that are evenly split between all possible values.

Intuitively, the impurity of a set of points is higher when the points have many different labels and lower when most points have the same label.

In [1]:
import numpy as np

In [8]:
first_coefficient = 16/30
negative_coefficient = -(first_coefficient)
logBase2_of_first_coeefficient = np.log2(first_coefficient)
first_term = negative_coefficient * logBase2_of_first_coeefficient

second_coefficient = 14/30
# negative_second_coefficient = -(second_coefficient)
logBase2_of_second_coefficient = np.log2(second_coefficient)
second_term = second_coefficient * logBase2_of_second_coefficient

result = first_term - second_term

In [9]:
result

0.9967916319816366